In [12]:
from hicstuff import hicstuff
import pandas as pd
import hicstuff.view as hcv
import hicstuff.io as hio
import numpy
import sys

todense = lambda x: hcv.sparse_to_dense(x, remove_diag=False)

Script de Gael et Vittore qui utilise la fonction todense de hicstuff
En input il faut une matrice (...mat.tsv) deja binnee en kb avec la fonction hicstuff rebin

#Editer et copier dans le terminal si besoin


hicstuff rebin -b 1kb -f /mnt/e/Science/HiC/Contact_files/Sparse/fragments_list_W303_710_DpnIIHinfI.txt -c /mnt/e/Science/HiC/Contact_files/Sparse/info_contigs_W303_710_DpnIIHinfI.txt /mnt/e/Science/HiC/Contact_files/Sparse/HB27_1_2_merged_trimmed.txt /mnt/e/Science/HiC/Contact_files/Rebin/HB27_1_2_merged_1kb

Utilise hicstuff ici: le chargement de la matrice + conversion prend quelques minutes max sur un PC.

Les coordonnees des chromosomes sont donnes par le fichier frag.tsv genere par hicstuff rebin

In [30]:
##change the input directory for S288c or other genomes

#input_dir = "/mnt/e/Science/HiC/Contact_files/Rebin/"  # 1st argument: sparce matrix file name and path
input_dir = "/mnt/e/Science/HiC/Contact_files/Rebin/S288c/"  # 1st argument: sparce matrix file name and path
#output_dir = "/mnt/e/Science/HiC/Contact_files/Dense/"  # 2nd argument: dense matrix file name and path
output_dir = "/mnt/e/Science/HiC/Contact_files/Dense/S288c/"  # 2nd argument: dense matrix file name and path

#samples = ["AD362_S288c_DSB_cutsite_q20", "AD363_S288c_DSB_cutsite_q20", "AD364_S288c_DSB_cutsite_q20",
#          "AD365_S288c_DSB_cutsite_q20", 
#          "AD368_S288c_DSB_cutsite_q20", "AD369_S288c_DSB_cutsite_q20"]

samples = ["AD255_S288c_DSB_chr3_rDNA_cutsite_q20_v312", "AD256_S288c_DSB_chr3_rDNA_cutsite_q20_v312"]  

In [21]:
sample = "SRX20747219_SRR24991179_S288c_DSB_chr3_rDNA_cutsite_q20_PCRfree"
inputcool_dir = "/mnt/e/Science/HiC/Contact_files/Cool/S288c/"
a, frags, chrom = hio.load_cool(str(inputcool_dir + sample + ".cool"))
dense_a = todense(a)

In [31]:
chrom_intersect_1 = ["chr5"] #specifies the primary chromosome(s) to make intersection
chrom_intersect_2 = ["chr2", "chr3"] #specifies the secondary chromosome(s) to make intersection
chromlist = ["chr2", "chr3", "chr4", "chr5", "chr11", "chr12"] #chromosomes to extract for intra analysis

for sample in samples:
    frags = pd.read_csv(str(input_dir + sample + "_1kb.frags.tsv"), sep = "\t")
    #frags = pd.read_csv(str(input_dir + sample + "_1kb.frag.tsv"), sep = "\t")
    
    sparse_mat = hio.load_sparse_matrix(str(input_dir + sample + "_1kb.mat.tsv"))
    dense_mat = todense(sparse_mat)
    dense_mat = dense_mat.astype(int) # https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html
    numpy.savetxt(str(output_dir + sample + "_dense_1kb.mat.csv"), dense_mat, fmt="%i")

    #Export individual chromosomes
    for chrom in chromlist:

        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)

        dense_mat2 = dense_mat[start:stop,start:stop]
        numpy.savetxt(str(output_dir + sample + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #export chromosome intersections    
    for chrom in chrom_intersect_1:
        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)
        
        for chrom_alt in chrom_intersect_2:
            start_alt = min(frags[(frags["chrom"]== chrom_alt)].index.values)
            stop_alt = max(frags[(frags["chrom"]== chrom_alt)].index.values)
            dense_mat2 = dense_mat[start_alt:stop_alt,start:stop]
            numpy.savetxt(str(output_dir + sample + "_" + str(chrom_alt) + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #remove the 2-micron and the mitochondria
    start = 0
    stop = max(frags[(frags["chrom"]== "chr16")].index.values)
    dense_mat2 = dense_mat[start:stop,start:stop]
    numpy.savetxt(str(output_dir + sample + "_chrs_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    print(str(sample) + " done")

AD255_S288c_DSB_chr3_rDNA_cutsite_q20_v312 done
AD256_S288c_DSB_chr3_rDNA_cutsite_q20_v312 done


For extraction of specific chromosomes

In [10]:
chromlist = ["chr12"] #chromosomes to extract for intra analysis

for sample in samples:
    frags = pd.read_csv(str(input_dir + sample + "_1kb.frasg.tsv"), sep = "\t")
    #frags = pd.read_csv(str(input_dir + sample + "_1kb.frag.tsv"), sep = "\t")
    
    sparse_mat = hio.load_sparse_matrix(str(input_dir + sample + "_1kb.mat.tsv"))
    dense_mat = todense(sparse_mat)
    dense_mat = dense_mat.astype(int) # https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html
    numpy.savetxt(str(output_dir + sample + "_dense_1kb.mat.csv"), dense_mat, fmt="%i")

    #Export individual chromosomes
    for chrom in chromlist:

        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)

        dense_mat2 = dense_mat[start:stop,start:stop]
        numpy.savetxt(str(output_dir + sample + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    print(str(sample) + " done")
    

In [4]:
print(sample)

AD355_S288c_DSB_cutsite_q20


# For Meiosis synthetic region

In [2]:
##change the input directory for S288c or other genomes

input_dir = "/mnt/e/Science/HiC/Contact_files/Rebin/Meiosis/"  # 1st argument: sparce matrix file name and path
output_dir = "/mnt/e/Science/HiC/Contact_files/Dense/Meiosis/"  # 2nd argument: dense matrix file name and path

samples = ["AD625_SK1_2019_Diploid_SynIVv1_SynIVv2_inversion_iterative_q20",
          "AD626_SK1_2019_Diploid_SynIVv1_SynIVv2_inversion_v2only_iterative_q20",
          "AD623_SK1_2019_Diploid_SynIVv1_SynIVv2_iterative_q20",
          "AD624_SK1_2019_Diploid_SynIVv1_SynIVv2_iterative_q20"]

In [3]:
chrom_intersect_1 = ["chr4_v1"] #specifies the primary chromosome(s) to make intersection
chrom_intersect_2 = ["chr2", "chr11", "SynIV_v2"] #specifies the secondary chromosome(s) to make intersection
chromlist = ["chr2", "chr3", "chr4_v1", "chr5", "chr11", "SynIV_v2"] #chromosomes to extract for intra analysis

for sample in samples:
    frags = pd.read_csv(str(input_dir + sample + "_1kb.frags.tsv"), sep = "\t")
    
    sparse_mat = hio.load_sparse_matrix(str(input_dir + sample + "_1kb.mat.tsv"))
    dense_mat = todense(sparse_mat)
    dense_mat = dense_mat.astype(int) # https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html
    numpy.savetxt(str(output_dir + sample + "_dense_1kb.mat.csv"), dense_mat, fmt="%i")

    #Export individual chromosomes
    for chrom in chromlist:

        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)

        dense_mat2 = dense_mat[start:stop,start:stop]
        numpy.savetxt(str(output_dir + sample + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #export chromosome intersections    
    for chrom in chrom_intersect_1:
        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)
        
        for chrom_alt in chrom_intersect_2:
            start_alt = min(frags[(frags["chrom"]== chrom_alt)].index.values)
            stop_alt = max(frags[(frags["chrom"]== chrom_alt)].index.values)
            dense_mat2 = dense_mat[start_alt:stop_alt,start:stop]
            numpy.savetxt(str(output_dir + sample + "_" + str(chrom_alt) + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #remove the 2-micron and the mitochondria
    start = 0
    stop = max(frags[(frags["chrom"]== "chr16")].index.values)
    dense_mat2 = dense_mat[start:stop,start:stop]
    numpy.savetxt(str(output_dir + sample + "_chrs_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #individual and intersection entre SynIV_v1 - SynIV_v2
    start_v1 = 2107
    stop_v1 = 2257
    start_v2 = min(frags[(frags["chrom"]== "SynIV_v2")].index.values)
    stop_v2 = max(frags[(frags["chrom"]== "SynIV_v2")].index.values)
    dense_mat2 = dense_mat[start_v1:stop_v1,start_v2:stop_v2]
    numpy.savetxt(str(output_dir + sample + "_SynIV_intersect_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    dense_mat2 = dense_mat[start_v1:stop_v1,start_v1:stop_v1]
    numpy.savetxt(str(output_dir + sample + "_SynIV_v1_dense_1kb.mat.csv"), dense_mat2, fmt="%i")    
    dense_mat2 = dense_mat[start_v2:stop_v2,start_v2:stop_v2]
    numpy.savetxt(str(output_dir + sample + "_SynIV_v2_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    print(str(sample) + " done")

AD625_SK1_2019_Diploid_SynIVv1_SynIVv2_inversion_iterative_q20 done
AD626_SK1_2019_Diploid_SynIVv1_SynIVv2_inversion_v2only_iterative_q20 done
AD623_SK1_2019_Diploid_SynIVv1_SynIVv2_iterative_q20 done
AD624_SK1_2019_Diploid_SynIVv1_SynIVv2_iterative_q20 done


For Corentin

In [9]:
chrom_intersect_1 = ["chr5_annot"] #specifies the primary chromosome(s) to make intersection
chrom_intersect_2 = ["chr2", "chr3", "chr6", "chr7"] #specifies the secondary chromosome(s) to make intersection
chromlist = ["chr2", "chr3", "chr5_annot", "chr7", "chr11"] #chromosomes to extract for intra analysis

for sample in samples:
    frags = pd.read_csv(str(input_dir + sample + "_1kb.frag.tsv"), sep = "\t")
    
    sparse_mat = hio.load_sparse_matrix(str(input_dir + sample + "_1kb.mat.tsv"))
    dense_mat = todense(sparse_mat)
    dense_mat = dense_mat.astype(int) # https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html
    numpy.savetxt(str(output_dir + sample + "_dense_1kb.mat.csv"), dense_mat, fmt="%i")

    #Export individual chromosomes
    for chrom in chromlist:

        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)

        dense_mat2 = dense_mat[start:stop,start:stop]
        numpy.savetxt(str(output_dir + sample + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #export chromosome intersections    
    for chrom in chrom_intersect_1:
        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)
        
        for chrom_alt in chrom_intersect_2:
            start_alt = min(frags[(frags["chrom"]== chrom_alt)].index.values)
            stop_alt = max(frags[(frags["chrom"]== chrom_alt)].index.values)
            dense_mat2 = dense_mat[start_alt:stop_alt,start:stop]
            numpy.savetxt(str(output_dir + sample + "_" + str(chrom_alt) + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #remove the 2-micron and the mitochondria
    start = 0
    stop = max(frags[(frags["chrom"]== "chr16")].index.values)
    dense_mat2 = dense_mat[start:stop,start:stop]
    numpy.savetxt(str(output_dir + sample + "_chrs_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    print(str(sample)+" is done")

AD516_SK1_2019_Corentin_iterative_q10 is done
AD517_SK1_2019_Corentin_iterative_q10 is done
AD518_SK1_2019_Corentin_iterative_q10 is done
AD519_SK1_2019_Corentin_iterative_q10 is done
AD520_SK1_2019_Corentin_iterative_q10 is done
AD521_SK1_2019_Corentin_iterative_q10 is done
AD522_SK1_2019_Corentin_iterative_q10 is done
AD523_SK1_2019_Corentin_iterative_q10 is done


In [8]:

    print(str(sample)+" is done")

AD516_SK1_2019_Corentin_iterative_q10 is done


## For Glabrata

In [2]:
##change the input directory for S288c or other genomes

#input_dir = "/mnt/e/Science/HiC/Contact_files/Rebin/"  # 1st argument: sparce matrix file name and path
input_dir = "/mnt/e/Science/HiC/Contact_files/Rebin/Other_genomes/"  # 1st argument: sparce matrix file name and path
#output_dir = "/mnt/e/Science/HiC/Contact_files/Dense/"  # 2nd argument: dense matrix file name and path
output_dir = "/mnt/e/Science/HiC/Contact_files/Dense/Other_genomes/"  # 2nd argument: dense matrix file name and path

samples = ["AT676_AT719_AT723_merged_CAGL_chr_MAT",
           "AT677_AT720_AT724_merged_CAGL_chr_MAT",
           "AT678_AT721_AT725_merged_CAGL_chr_MAT",
           "AT679_AT722_AT726_merged_CAGL_chr_MAT"]


In [4]:
chrom_intersect_1 = ["B"] #specifies the primary chromosome(s) to make intersection
chrom_intersect_2 = ["E", "C"] #specifies the secondary chromosome(s) to make intersection
chromlist = ["B", "C", "E", "K"] #chromosomes to extract for intra analysis

for sample in samples:
    frags = pd.read_csv(str(input_dir + sample + "_1kb.frag.tsv"), sep = "\t")
    #frags = pd.read_csv(str(input_dir + sample + "_1kb.frag.tsv"), sep = "\t")
    
    sparse_mat = hio.load_sparse_matrix(str(input_dir + sample + "_1kb.mat.tsv"))
    dense_mat = todense(sparse_mat)
    dense_mat = dense_mat.astype(int) # https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html
    numpy.savetxt(str(output_dir + sample + "_dense_1kb.mat.csv"), dense_mat, fmt="%i")

    #Export individual chromosomes
    for chrom in chromlist:

        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)

        dense_mat2 = dense_mat[start:stop,start:stop]
        numpy.savetxt(str(output_dir + sample + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    
    #export chromosome intersections    
    for chrom in chrom_intersect_1:
        start = min(frags[(frags["chrom"]== chrom)].index.values)
        stop = max(frags[(frags["chrom"]== chrom)].index.values)
        
        for chrom_alt in chrom_intersect_2:
            start_alt = min(frags[(frags["chrom"]== chrom_alt)].index.values)
            stop_alt = max(frags[(frags["chrom"]== chrom_alt)].index.values)
            dense_mat2 = dense_mat[start_alt:stop_alt,start:stop]
            numpy.savetxt(str(output_dir + sample + "_" + str(chrom_alt) + "_" + str(chrom) + "_dense_1kb.mat.csv"), dense_mat2, fmt="%i")
    